## 1. Dataset preparation

In [ ]:
import json
from dataset_dic import formatted_data
ds_formatted = formatted_data()

In [ ]:
ds_formatted[0]

In [ ]:
len(ds_formatted)

In [ ]:
import random
random.shuffle(ds_formatted)

In [ ]:
ds_train = ds_formatted[:12]
ds_val = ds_formatted[12:]

In [ ]:
# preprocess content: dict to str
for i in range(len(ds_formatted)):
    init_dic = ds_formatted[i]['messages'][2]['content']
    dic2json = json.dumps(init_dic, indent=4)
    ds_formatted[i]['messages'][2]['content'] = dic2json

In [ ]:
import json

with open('train.jsonl', 'w') as f:
    for line in ds_train:
        json.dump(line, f)
        f.write('\n')

with open('val.jsonl', 'w') as f:
    for line in ds_val:
        json.dump(line, f)
        f.write('\n')

In [ ]:
ls

In [ ]:
!head -n 2 train.jsonl

## 2. Upload data

In [ ]:
import openai

openai.api_key = 'my_api_key_here'

# pip install openai==0.28 # download the old version and then restart kernel

In [ ]:
# Upload training data
train = openai.File.create(
  file=open("train.jsonl", "rb"),
  purpose='fine-tune'
)
train

In [ ]:
train_id = train['id']

In [ ]:
# Upload validation data
val = openai.File.create(
  file=open("val.jsonl", "rb"),
  purpose='fine-tune'
)
val

In [ ]:
val_id = val['id']

## 3. Fine tune

In [ ]:
# Create a fine-tuned model
response = openai.FineTuningJob.create(
    training_file=train_id,
    validation_file=val_id,
    model="gpt-3.5-turbo"
    )
response

In [ ]:
job_id = response['id']

In [ ]:
# Retrieve the state of a fine-tune
response = openai.FineTuningJob.retrieve(job_id)
response

In [ ]:
# List up to 10 events from a fine-tuning job
response = openai.FineTuningJob.list_events(id=job_id, limit=10)
response

In [ ]:
events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

In [ ]:
response = openai.FineTuningJob.retrieve(job_id)
response

In [ ]:
model_id = response["fine_tuned_model"]

## 4. Inference

In [ ]:
model_id

In [ ]:
test_messages = [
    {"role": "system", "content": "You are a robot assistant that can extract keywords from user's demands and help us save the keywords and corresponding parameters into a dictionary format."},
    {"role": "user", "content": "move the box to (10,10) point"}
  ]

In [ ]:
completion = openai.ChatCompletion.create(
  model=model_id,
  messages=test_messages
)

In [ ]:
print(completion.choices[0].message)

In [ ]:
gpt_reply = completion.choices[0].message
reply_dict = json.loads(gpt_reply['content'])
reply_dict
# type(gpt_reply['content'])

In [ ]:
reply_dict['what']

In [ ]:
completion1 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=test_messages
)

In [ ]:
print(completion1.choices[0].message)